In [1]:
!pip install --upgrade astrapy
!pip install langchain langchain-openai google-cloud-secret-manager

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 24.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.4 MB/s eta 0:00:00
  Created wheel for bson: filename=bson-0.5.10-py3-none-any.whl size=11978 sha256=253a4999b4eea850e201906566e79ae9ac6a6c17a6ebb7587c78b228bc918674
  Stored in directory: /Users/diego.collado/Library/Caches/pip/wheels/36/49/3b/8b33954dfae7a176009c4d721a45af56c8a9c1cdc3ee947945
Successfully built bson
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 14.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [3]:
from astrapy.db import AstraDB
import langchain
from langchain_openai import OpenAIEmbeddings
import json
from google.colab import userdata
import os

token = userdata.get('DATASTAX')
url_datastax = userdata.get('URL_DATASTAX')
secret_string= userdata.get('OPENAI_TOKEN')

# Configurar el motor de OpenAI
embeddings = OpenAIEmbeddings(api_key=secret_string, model="text-embedding-3-large")

def get_embedding(text):
    query_result = embeddings.embed_query(text)
    return query_result

# Initialization
db = AstraDB(
  token=token,
  api_endpoint=url_datastax)

print(f"Connected to Astra DB: {db.get_collections()}")

# Create a collection
collection = db.create_collection("vector_movies", dimension=3072, metric="cosine")
# Procesar cada línea y obtener los embeddings
# Leer el documento de texto

documents = []
with open('combined_info_movies.csv', 'r') as file:
    lines = file.readlines()

if os.path.exists('documentos.json'):
    with open('documentos.json', 'r') as file:
        documents = json.load(file)
    for document in documents:
        res = collection.upsert_one(document)

else:
    for index, line in enumerate(lines):
        # Extraer el texto de cada línea

        # Obtener el embedding para el texto
        vector = get_embedding(line)

        # Crear el documento
        document = {
            "_id": str(index + 1),
            "text": line,
            "$vector": vector
        }

        # Insertar el documento en la base de datos
        # Reemplaza 'collection' con tu objeto de colección de la base de datos
        # res = collection.insert_one(document)  # Usamos insert_one para insertar un solo documento
        res = collection.upsert_one(document)  # Aqui usamos upsert si existe se actualiza si no, se crea
        # Añadir al documento JSON
        documents.append(document)

        # Guardar los documentos en un archivo JSON
        with open('documentos.json', 'w') as file:
            json.dump(documents, file)
        print(documents)





Connected to Astra DB: {'status': {'collections': ['vector_movies']}}


In [4]:
# Perform a similarity search
pelicula_parecida = "It was a comedy where two old pals connected via some social network and one lured the other into espionage"
embedding_busqueda = get_embedding(pelicula_parecida)


results = collection.vector_find(embedding_busqueda, limit=2, fields={"text", "$vector"})

for document in results:
    print(document['text'])

"Title: Central Intelligence. Overview: After he reunites with an old pal through Facebook, a mild-mannered accountant is lured into the world of international espionage. Genres: Action,Comedy"

"Title: I Spy. Overview: When the Switchblade, the most sophisticated prototype stealth fighter created yet, is stolen from the U.S. government, one of the United States' top spies, Alex Scott, is called to action. What he doesn't expect is to get teamed up with a cocky civilian, World Class Boxing Champion Kelly Robinson, on a dangerous top secret espionage mission. Their assignment: using equal parts skill and humor, catch Arnold Gundars, one of the world's most successful arms dealers. Genres: Action,Adventure,Comedy,Thriller"



# query = [-0.0036464269346367442, 0.01719093074088834, -0.0366543545749403, -0.029437254045787793, 0.02635352031594596, -0.03179436061606439, -0.03337167454821107, 0.04586611149120767, -0.0401594326223954, 0.029738538750384857, -0.017740332808872716, -0.03707569702990848, -0.005901628890989831, -0.04965875117530822, -0.006242789101494209, 0.019051804447035393, -0.02257860369979702, 0.008205568120056673, 0.0668851261894547, -0.00642444589138552, 0.04069110975978308, -0.024687594769331384, 0.020150606720359062, -0.016721281547363676, 0.03916696689413632, 0.009782880189558273, -0.03771371257500583, -0.014390758647691957, -0.01700484318400913, 0.029773983023642998, 0.021993757358554505, 0.002189849889511285, 0.032060201047403314, -0.0236951290410723, 0.01986704508371361, 0.002228618052917576, 0.04792193521953228, -0.00442843681890569, 0.008985362620831668, -0.0019860399631723663, 0.013743883431263884, 0.011856426054511955, -0.0014997758591043353, 0.03831628198419996, -0.019477146902003573, 0.03462997698251891, -0.01513510794388627, -0.03308811291156562, -0.025839564383864803, 0.013832496908376858, 0.008941056813597721, 0.000387405176787437, 0.010163916837292506, -0.0018586586718990536, -0.04501542658127131, 0.037359258666553936, 0.002578639564567124, 0.019016360173777253, 0.041151898350654484, 0.01513510794388627, 0.012264045441528523, -0.012503301270940029, -0.008861304870460553, 0.005214877702993174, 0.025591447020477495, -0.05671234968083147, -0.014709765488918091, 0.004745228509468508, -0.0063668473175265946, 0.03475403659553511, -0.011342470122430802, 0.0401594326223954, 0.005396534492884484, 0.011546279815939087, 0.024598981292218408, 0.018839133219551305, 0.011909593395721708, -0.01713776339967859, 0.013646409351497645, 0.01746563177488053, 0.004572433253553002, 0.02396096760976614, 0.026601635816688188, 0.015649064807289965, 0.027399153385414793, -0.010633566030817172, -0.018821412014244775, -0.07184745855604029, -0.013823635374401053, -0.051537347949406884, 0.055578103134249664, -0.039627755485007715, -1.7281275459293192e-05, 0.01805933871877631, -0.020611395311230463, 0.022011480426506115, 0.04072655589568631, 0.02163930531274769, -0.03588828314211692, -0.0028843545704908204, -0.056393341908282796, -0.004311024520524524, 0.019991102834084725, -0.035179379050503286, 0.06986252709952212, 0.02128485326694087, 0.04023032489420184, -0.015356640705346165, 0.003766054383681227, 0.01917586406005159, -0.025006602541880057, -0.005378811890594143, -0.050828443857793254, 0.016109851535516282, -0.006322541044631378, -0.0011475389639608359, -0.03657946602842402, 0.019548037311164935, -0.01434645190913547, -0.040655667349170026, -0.026140849088461867, 0.0032365914656343198, -0.004771812645734654, -0.018378346491324984, -0.02794855545339917, 0.0026295922207748303, 0.000439742243952057, -0.0007515494089649724, -0.01105890848578535, 0.011564002883890697, -0.006433307425361325, -0.02734598604420504, -0.014018584465256072, 0.008422669648867395, 0.003134686618880178, 0.008170122915475992, 0.037820047257425334, 0.03496670968566428, -0.0027780189567487754, 0.04083289057810581, -0.005498439339638626, -0.03170574713895141, -0.0027270665333717047, 0.03624273705056882, -0.00205360741350812, -0.02484709865560572, -0.019742987333342496, 0.05610978027163734, -0.022950778813555447, -0.029277750159513456, 0.0442356339431414, 0.011900731861745902, 0.0005062020316446643, 0.026300352974736207, -0.010500645815147711, 0.011741228906794106, -0.0029818291159183297, 0.023145726973087925, 0.019246754469212955, -0.019955658560826584, 0.02575095090675183, 0.0333893957535176, -0.0032055770280415407, 0.02642441072510732, 0.017350434627162677, -0.037217477848231206, -0.002804602860184287, -0.017873251161897096, 0.010704456439978534, 0.035693334982584445, 0.010474062144542834, -0.006592810845974392, 0.00831633403512535, 0.0007842254423858735, 0.035516108028358494, 0.005449702299755507, 0.017128902797025326, -0.004322101205163645, -0.01767830300236462, -0.014497094261434001, 0.007868838210878928, -0.040442994259040854, -0.0013635332666858523, -0.017430185638977306, 0.0541602949510224, 0.002926445925553356, 0.03193614143438712, 0.0014831608321456526, 0.02096584735703728, -0.02390780026855639, 0.02954359059085238, -0.006411154056083081, 0.013929970988143097, 0.030801894887805305, 0.027097870543462813, -0.005595913885066135, 0.005259184441549661, -0.016180741944677647, -0.05752758845486461, -0.02282672106318433, 0.03767826643910261, -0.01500218865953935, -0.028108059339673506, 0.024634427428121632, 0.032698212867210495, 0.03067783713743419, -0.016145297671419506, 0.003500215349326115, -0.006995999931664327, -0.010863959394930331, -0.0035024307328200663, -0.027204205225882315, -0.005680096129529936, 0.018910023628712667, -0.0006850895921685356, 0.05213991735860101, -0.012999534135069572, -0.014860407841216623, -0.004789535248024995, -0.015525006125596309, 0.03462997698251891, 0.017687163605017882, 0.020451891424956126, -0.003980940994659268, 0.0008379470951303836, -0.013105869748811616, -0.0169871219787026, 0.027381432180108262, -0.020912678153182446, 0.047496592764564104, 0.011998206872834682, -0.009330954063985219, -0.012955227396513084, -0.030536056319111462, -0.04703580417369271, -0.04632690008207907, 0.007368175045422754, -0.05455019127008736, 0.017722609740921106, 0.022454545949425905, 0.022933055745603836, 0.028586569135851437, -0.02084178774402108, 0.0027913110248818486, 4.413345071825905e-05, 0.004359762026068913, 0.020487335698214267, 0.023092559631878173, 0.010925988270115889, -0.004940177134662257, -0.0034138177213683626, 0.0082631666939156, 0.0009708666705155984, 0.00938412140519497, -0.006579518545010684, 0.023074836563926563, 0.027381432180108262, 0.002558701811613467, -0.010801930519744775, -0.03404513250392148, 0.013309679442319899, -0.031050012251192617, -0.0020458536876946077, 0.02034555487989154, -0.04228614489723629, -0.005959227464848757, 0.0370048084833922, -0.022029203494457725, -0.01309700821483581, 0.03170574713895141, 0.013380569851481262, 0.010686733372026923, -0.013362847714852192, 0.011590586554495574, -0.012317213714060816, -0.018236565673002258, -0.0031834236587632973, -0.035374327210035764, -0.05196269040437506, -0.008152399847524382, 0.03542749455124552, 0.019459425696697043, 0.03432869414056693, -0.04512176498898098, -0.03245009736646827, 0.017332711559211067, -0.019902491219616833, 0.017394741365719165, 0.002297293311415623, -0.05713769213579965, 0.00018248750049388356, 0.03987587098574995, 0.010465200610567028, 0.029171415477093954, -0.011750090440769911, 0.026495301134268685, 0.00906954579661801, 0.023039392290668422, -0.02647757806631707, 0.014532539466014684, -0.005950365930872951, -0.016145297671419506, -0.026601635816688188, -0.0248116525197025, 0.040478440394944075, 0.02683203011212389, 0.022206430448683676, 0.038032720347554505, -0.052494367541762746, 0.0007216424884961929, -0.02279127492728111, -0.03533888479942271, -0.04483820335233553, 0.0041227218129819936, -0.025361054587686876, 0.008369502307657643, 0.005649081691937158, -0.016287078489742233, 0.04221525635072001, -0.03468314804901883, 0.01544525418245914, -0.010748762247212482, -0.042959606578236866, 0.006521920436813029, -0.04653957317220824, 0.03330078227640462, 0.005897198124001929, 0.01186528665716522, 0.024244529246411593, -0.0526715944959887, -0.06369505591454828, -0.013859080578981733, 0.003068226743876082, -0.026956089725140086, -0.01565792540994323, -0.00375497793187274, 0.04802826990195179, -0.015994654387798433, -0.05979608527331839, -0.03542749455124552, -0.018466958105792877, -0.011200689304108077, -0.0255028354060096, -0.012600775350706268, 0.009198033848315757, 0.015241444488950855, 0.015285750296184801, -0.02521927376936415, 0.014186948954183674, -0.021036737766198643, 0.0009442827670800871, 0.013353986180876387, -0.0040363244178548766, 0.05128923244866465, 0.02009743937914931, -0.02846251138548032, -0.05837827336480097, 0.008914472211670304, -0.0017888758380693486, -0.032166535729822816, -0.009277785791452926, -0.002833402147113749, -0.0009332061406486242, 0.016854201763033136, -0.04253426412326869, 0.04990686667605045, -0.00047241830647678747, 0.033655236184856524, 0.009277785791452926, 0.024935712132718696, 0.0002489472881864909, 0.021019014698247032, -0.05185635572195556, -0.011014601747228863, -0.02839162097631896, -0.029454977113739407, 0.02215326124482884, -0.026814307044172276, 0.0019638865938941224, 0.008378363841633448, -0.01852012730964771, 0.016402275637460082, 0.008139108012221943, -0.01707573359317049, 0.02755865727168913, 0.004957899736952597, 0.02908280199998098, 0.03482492886734156, -0.03048288897790171, -0.021834255334925248, -0.024669871701379773, -0.0038059303552498112, 0.020186052856262286, -0.020239220197472037, 0.04980053199363094, 0.017332711559211067, -0.01710231726377537, 0.02254315942653888, 0.0022751399421373796, 0.02821439402209301, -0.03849350893842591, 0.04781560053711278, 0.006894095084910186, -0.062135466912998295, 0.006676993090438193, -0.008958778950226792, -0.007700473721951326, 0.008085940671012192, -0.0070491677385353495, 0.028249840157996232, -0.005688957663505742, -0.015223721420999245, -0.03789093952923178, -0.022472269017377516, 0.011333608588454997, -0.030305662023675763, 0.0033650804486546077, 0.02084178774402108, -0.0324678185717748, -0.026495301134268685, 0.05136012099518093, 0.0059193514932801726, -0.005458563833731312, 0.041222790622460925, -0.028551122999948216, 0.008879027007089623, -0.027204205225882315, 0.030199325478611178, -0.004211334824433697, -0.006672562323450291, -0.03356662270774355, 0.024988879473928447, 0.007120058147696713, -0.011608308691124645, 0.021036737766198643, -0.017403601968372432, -0.012485578202988418, -0.0473902580821446, -0.006995999931664327, 0.0004884794293543236, -0.001342487705569902, -0.0064421684936758605, -0.033956519026808504, -0.037820047257425334, 0.007168795653241103, -0.01917586406005159, -0.005498439339638626, -0.029224582818303705, 0.015303473364136413, 0.043810291488173225, -0.04310138739655959, -0.012086819418625116, -0.06167468204741705, -0.0008368394615910667, 0.00360655096306816, 0.04466097639810958, 0.03136901816109621, -0.007248547130717001, 0.03675669298264997, -0.03092595263817642, 0.0025542710446255644, 0.010793068985768968, -0.012122264623205797, 0.013575518942336282, -0.021745641857812272, 0.005276907043840001, -0.03927330157655582, -0.026530745407526826, -0.019796154674552247, -0.0002227787400522662, 0.005135126225517275, -0.023003946154765198, -0.014319868238530594, 0.014435065386248445, -0.06358872123212878, 0.02257860369979702, 0.04317227594307587, -0.006114300118473921, -0.014479372124804931, 0.01138677686098729, -0.02364196169986255, -0.019370812219584067, 0.0005272476509682734, 0.006087716447869045, -0.013017256271698642, 0.04303049512475315, 0.025431944996848237, 0.025396498860945017, -0.004887009327791234, 0.05026132058450235, 0.005870614453397053, -0.012671665759867631, -0.01163489329305206, -0.012955227396513084, -0.0008623157314872611, -0.02892329811370664, 0.004714214071875729, -0.023074836563926563, 0.03845806280252268, 0.018856856287502916, 0.007483371727479334, 0.009455011814356333, -0.02887013077249689, 0.02956131179615891, 0.013903387317538221, 0.015276889693531538, 0.05710224599989643, 0.023074836563926563, -0.029720815682433246, 0.009623377234606477, -0.015932626443935418, 0.03369067859546958, -0.00851128312598037, -0.038918851393394086, -0.004891440094779136, 0.03264504552600075, -0.01967209692418113, 0.01853784851495424, -0.0469294694912732, -0.020700006925698358, -0.001025696015007084, 0.010287974587663621, 0.01546297631908821, 0.004647754429702268, 0.015870596637427316, -0.01702256625196074, -0.007456788056874458, 0.0008235475098733112, -0.008653063944303097, -0.050013205083760115, 0.01696939891075099, 0.02002654896998795, -0.022614049835700242, -0.0012871045152049283, 0.017474492377533794, 0.035179379050503286, 0.01207795788464931, -0.014638875079756728, 0.004944607901650159, -0.017385878900420818, -0.019512593037906794, 0.036207290914665596, -0.02167475144865091, -0.019937935492874974, 0.001386794211295754, 0.038068166483457726, 0.020221497129520427, 0.03489581741385783, -0.017474492377533794, -0.007465649125188993, -0.015117385807257199, -0.005644650924949255, 0.005883906288699491, -0.027470043794576158, -0.000671243852784951, 0.0027314970675289723, 0.024474923541847295, 0.010491784281171904, -0.027877665044237807, -0.01746563177488053, 0.04902073563021087, 0.013380569851481262, 0.0658572143252924, 0.026619358884639798, 0.02367740597312069, 0.036614912164327246, -0.0036929488238565474, 0.0009536979140987454, -0.05206902508679457, 0.007735918926532008, 0.015356640705346165, 0.004492681310415834, -0.025361054587686876, 0.014284423033949913, -0.016641530535549048, 0.004344254574441889, -0.00530792148143278, -0.004621170293436123, 0.03657946602842402, -0.0004624493426884708, 0.011723505838842494, -0.012095680952600921, 0.02210009390361909, -0.002658391507704293, -0.0023659683370830346, -0.019813877742503858, 0.0200797163111977, -0.02084178774402108, -0.018910023628712667, 0.012822308112166163, -0.0001786106666909568, -0.0012272906742673694, 0.06458119068567802, -0.023836909859395027, 0.0039410650230906836, 0.006451030027651666, 0.035179379050503286, -0.0182720099462604, -0.025059769883089812, -0.017935280968405194, -0.024545813951008657, 0.01657950072904095, 0.014417342318296833, -0.019477146902003573, 0.012122264623205797, 0.022684940244861604, 0.012343797384665692, 0.033956519026808504, 0.03193614143438712, -0.002485596018958153, -0.010571536224309072, 0.005033220913101863, 0.03322989372988834, -0.02486482172355733, 0.026743416635010914, -0.014957881920982862, 0.027664993816753716, -0.02644213379305893, 0.012219738702972036, -0.02210009390361909, 0.04887895481188814, -0.04239248330494596, -0.001238367358906491, -0.026052235611348895, 0.019477146902003573, 0.007297284170600121, -0.0170934566611221, -0.00444615942119603, 0.006185190527635284, 0.039698644031524, 0.019034083241728863, -0.03588828314211692, 0.00859546537044417, -0.012113403089229991, 0.004709783304887826, -0.010988018076623987, 0.006694715692728534, 0.03259187818479099, 0.02885240770454528, 0.007102335545406372, -0.013336263112924776, -0.0047186448388636315, -0.016012377455750043, -0.029206859750352095, -0.010881682462881943, -0.02913596934119073, 0.005321213316735218, -0.003972079926344733, 0.01605668419430653, 0.018980914037874032, -0.009060684262642204, -0.033956519026808504, -0.008737246188766897, -0.0002489472881864909, -0.007514386165072113, -0.023482457813588212, 0.009711989780396911, 0.011342470122430802, -0.00025060876759929563, 0.019423979560793822, 0.025875010519768028, -0.021036737766198643, 0.014302146101901524, -0.014293284567925718, -0.005945935629546319, 0.008059357000407314, 0.021462080221166822, -0.012680527293843438, 0.005285768112154536, -0.005923782260268075, 0.03136901816109621, -0.0008074863869957751, -0.0027890956413878973, 0.014284423033949913, 0.012565330146125587, -0.017049149922565614, -0.027753605431221608, -0.02210009390361909, -0.00721310192613632, -0.007332729840842073, -0.0569604651815737, -0.0009071760830866008, 0.0182720099462604, -0.024209084973153452, -0.019423979560793822, -0.004971192037916306, -0.014886991511821499, 0.023128005767781394, 0.016756727683266897, 0.010146193769340894, -0.014178087420207867, -0.007501094329769675, 0.007988466591245951, 0.033548897777146855, -0.01919358526535812, -0.009034099660714787, 0.01503763386412003, 0.003613196880719379, -0.011519696145334211, 0.04639779235388552, -0.01805933871877631, -0.005826307714840565, -0.00010093579837447189, 0.002946383911337648, -0.005644650924949255, 0.011776674111374787, -0.0036021204289108924, -0.014700904886264826, 0.00925120212084805, 0.0031324712353862265, 0.001758968975808228, 0.011085492156390226, -0.009800603257509885, -0.01758082892259838, 0.009782880189558273, -0.011555141349914892, 0.022897611472345696, -0.0038325142586853223, -0.004563571719577197, -0.021905145744086613, 0.026814307044172276, 0.04264059880568819, -0.028356174840415735, -0.01565792540994323, -0.016606084399645827, -0.022454545949425905, 0.037961828075748064, -0.011422222065567972, 0.008958778950226792, 0.0007836716547200445, 0.006752314266587459, -0.03337167454821107, 0.023447011677684988, 0.015312334898112219, -0.03179436061606439, -0.026495301134268685, 0.0268497531800755, 0.01296408893048889, 0.007456788056874458, 0.008471407620073056, 0.06894094991777931, 0.002051392030014168, -0.007913144949435416, -0.0010002198033185487, -0.0038014998210925436, 0.03218425693512934, 0.01926447567451948, 0.03473631539022858, -0.0029906904170635, 0.02210009390361909, 0.0033938797355840705, 0.024226806178459983, -0.0012804584811383917, -0.03223742800162926, -0.022188707380732062, -0.01276913983963387, 0.010793068985768968, -0.039415082394878544, -0.011298163383874316, -0.031546245115322155, 0.03120951427482187, 0.020469614492907736, 0.03143991043290265, 0.012317213714060816, 0.028161226680883257, 0.009277785791452926, -0.0017279544218001316, -0.01598579378514517, -0.0010079734127167435, -0.01565792540994323, 0.019565760379116545, 0.025680060497590466, -0.014417342318296833, -0.00800175842654839, 0.002132251548482995, 0.005232600305283514, -0.005294629646130342, 0.00547185566903375, -0.03159941245653191, 0.00011381238023902629, -0.010128471632711824, 0.049623305039405, -0.01922903140126134, -0.004873717492488796, 0.008848013035158114, -0.013407153522086138, -0.006003533737743974, -0.014293284567925718, -0.018608738924115603, -0.002977398348930427, 0.05667690354492825, 0.013921109454167292, 0.022401378608216154, 0.032946332093242894, -0.005214877702993174, -0.05189180185785878, 0.005276907043840001, -0.010217084178502258, -0.004576864020540906, -0.04019487875829862, 0.029827152227497832, -0.025166104565509315, -0.017527661581388628, 0.0016969398677920352, -0.0062826650730627934, -0.026318074180042735, -0.004510403912706175, -0.009720851314372716, 0.011493112474729334, 0.04572433067288494, 3.963356941148787e-05, -0.032839993685533225, 0.010465200610567028, 0.03704025461929542, -0.002888785337478723, 0.0037815618353082515, -0.026495301134268685, 0.0402657673048149, 0.0030172743204990114, 0.0011342470122430804, 0.04583066908059461, -0.0031590551388217375, -0.0048559948901984555, 0.007642875148092401, -0.002665037425355512, -0.012999534135069572, 0.009756296518953397, 0.020416445289052905, 0.010793068985768968, -0.02530788538383204, -0.0020380999618810954, -0.022206430448683676, -0.018644185060018827, -0.006295956908365231, 0.03177663941075786, 0.035143932914600065, -0.004674338100307145, 0.0021355745073086047, 0.021426634085263598, 0.06022142772828657, 0.006575088243684052, -0.018201119537099037, -0.03044744284199849, -0.019087250582938618, 0.005166140663110054, -0.0004967868846260061, -0.022011480426506115, -0.027221928293833925, 0.034470474958889656, -0.010394310201405665, 0.027470043794576158, 0.0368984738009727, 0.007004861465640133, -0.014399620181667763, -0.017049149922565614, -0.023074836563926563, -0.0019816091961844633, -0.03092595263817642, 0.04852450462872641, 0.012840030248795233, 0.01651747278517793, -0.0045901558558433435, -0.04459008412630314, -0.005219308469981076, -0.033584343913050076, 0.0025609169622767833, 0.015347780102692901, -0.03789093952923178, -0.004669907333319242, 0.0205405049020691, -0.0017334927641196925, -0.013123591885440686, 0.04306594126065637, 0.00026002391461795387, -0.014833824170611748, -0.034027411298614946, -0.014169225886232062, -0.002698267246442242, 0.001250551618877271, -0.003952141707729805, -0.023163450041039535, -0.010314558258268497, 0.0016271570339623303, -0.015649064807289965, -0.0007665028400955327, -0.0023681837205769864, -0.0017966295638828609, -0.0004610647571085806, 0.016180741944677647, 0.004754090043444313, 0.007895422812806346, -0.0135400737377556, -0.013247650567134343, -0.05710224599989643, -0.007089043710103934, 0.02598134520218753, -0.009304369462057802, 0.016485878900420818, 0.0028644168175371632, 0.0008916688060825527, -0.01967209692418113, -0.021905145744086613, 0.015365502239321972, -0.0004854333352577992, -0.00021156365215243124, -0.015303473364136413, 0.03675669298264997, -0.02433314272352457, -0.008201137818730041, 0.0017988448309614947, -0.01812136852528441, 0.03450592109479288, 0.002840048064764968, 0.03044744284199849, 0.0005147864389569202, 0.03260959939009753, -0.022454545949425905, 0.011138660428922519, -0.0031191791672531533, -0.005458563833731312, 0.015666786012596495, 0.018147952195889286, 0.012910920657956597, 0.0071865182555314435, 0.0016105421234189651, -0.013380569851481262, -0.004244564878351063, -0.0013701791843370712, 0.006455460328978298, 0.026211739497623232, -0.05274248676779514, 0.01588831970537893, 0.026140849088461867, 0.025768673974703442, 0.01657950072904095, 0.0056978187318202774, -0.0021045598368851907, 0.019831598947810388, 0.025999068270139144, -0.025662337429638856, 0.000505371291938262, 0.012414687793827055, 0.0336197900489533, -0.017244098082098095, 0.0005552161399836748, -0.002155512260262262, -0.016889646036291277, -0.036437685210101295, 0.015161692545813687, 0.0024900265531154204, 0.001364640958432828, 0.02075317612955319, -0.03810360889407079, 0.012458994532383543, 0.011413360531592165, 0.015950347649241945, -0.03588828314211692, -0.0339742439574052, 0.0021455433837854333, 0.01296408893048889, 0.03344256309472735, -0.019831598947810388, -0.016916229706896154, -0.012033652077415363, -0.007013722533954668, 0.006344693948248351, -0.008852443336484748, 0.004918024231045283, -0.007687181886648888, 0.012795723510238747, 0.0053034907144448775, 0.003174562357618127, 0.02777132849917322, 0.004891440094779136, 0.007487802494467237, 0.014603429875176047, 0.017855528093945486, -0.010376588064776595, -0.00016130343425381721, 0.018147952195889286, 0.03757193175668311, -0.0030239202381502303, -0.004980053106230841, 0.01424897782936923, -0.005520592708916869, -0.0052503229075738555, 0.011714645236189229, -0.005976950067139097, 0.017758054014179247, 0.036366792938294853, -0.0013380570549973167, 0.00011083553870455996, 0.018839133219551305, -0.0007925329558652149, -0.020008825902036335, -0.006389000686804838, 0.019069527514987004, -0.0020990214945656297, 0.013150176487368104, -0.03179436061606439, 0.016685837274105535, -0.008657494245629728, -0.0034138177213683626, 0.003486923281193042, -0.00934867620061429, -0.011803257781979663, 0.0005959227348433439, 0.028125780544980036, -0.03842261666661946, 0.05532998763350743, -0.01607440726225814, -0.0010683410413201313, 0.04313682980717265, 0.003092595263817642, 0.00926892425747712, 0.015959210114540292, 0.009774018655582467, -0.02913596934119073, 0.031475352843515714, 0.010633566030817172, -0.02486482172355733, 0.01455912313661956, 0.013061563010255128, -0.007585276574233476, 0.011989345338858877, 0.0288878538404485, 0.009189173245662492, 0.018998637105825642, -0.003314128025277537, -0.014913576113748916, 0.0005682311396608571, -0.01400086139730446, 0.010544952553704197, 0.018103645457332798, 0.019831598947810388, -0.010412033269357277, 0.019937935492874974, -0.04572433067288494, -0.0012394749342381493, 0.025768673974703442, 0.01986704508371361, -0.025361054587686876, 0.016801034421823385, 0.01620732561528252, 0.016322522763000374, 0.008227721489334917, 0.00934867620061429, 0.015737676421757857, 0.004215765591421601, 0.005188294032388297, -0.04746114662866088, 0.01746563177488053, 0.012565330146125587, 0.011315886451825926, -0.005201585867690735, 0.0066016719142889275, -0.008289750364520475, -0.015285750296184801, -0.0031413325365313967, -0.007992896892572585, 0.000333129678169439, -0.02415591576929862, -0.012467856066359348, -0.014931298250377987, 0.020593672243278852, -0.006619394516579268, -0.007022584067930474, -0.0003303605070096585, 0.011838702986560345, 0.006765606101889897, -0.0041980429891312595, -0.009676544575816228, -0.009880354269324513, 0.0340805786398247, 0.01919358526535812, -0.019530316105858404, 0.018927746696664277, -0.001289319782283562, 0.002773588422591508, 0.0006452137370152689, -0.007567553971943135, 0.02116079551656976, 0.007372605346749387, 0.05618067254344379, 0.007319437539878364, -0.0067921902381560435, 0.005954796697860854, 0.020451891424956126, -0.03406285370922801, 0.017882113627195443, -0.031227237342773485, -0.0073770361137372896, -0.0018387206861147614, 0.0015629126588675036, 0.002538763825829175, 0.014169225886232062, -0.02389007720060478, 0.03330078227640462, -0.016978259513404252, 0.029029634658771227, 0.01501991172749096, 0.008303042199822913, -0.005117403623226934, 0.007775794898100592, 0.016623807467597437, 0.0111741056335032, 0.007899853114132977, -0.00934867620061429, 0.009933522541856804, 0.006570657476696148, -0.0020657916734789, 0.012272906975504328, -0.007851116074249858, 0.011218411440737147, 0.029472700181691017, -0.0216570283806993, -0.007518816932060015, 0.017217514411493218, 0.01061584296286556, 0.0056978187318202774, -0.011838702986560345, -0.012574191680101392, 0.005724402868086424, -0.0012217523319478084, -0.006251650635470014, 0.010146193769340894, 0.006216205430889333, 0.013531212203779794, 0.01194503860030239, -0.001622726383389745, -0.0043664079437201325, 0.011280441247245246, 0.02975626181833647, -0.002735927834516875, 0.0038280837245280547, 0.01322992749918273, -0.01611871400081463, 0.010181638973921577, -0.017199793206186687, -0.016570640126387686, -0.010110748564760214, 0.003322989326422707, 0.018927746696664277, 0.022525436358587267, 0.015347780102692901, 0.00749223326145514, -0.028533401794641686, 0.0330526667756624, -0.024085025360137256, 0.0028356175306077005, -0.010677871838051118, 0.0034116023378744113, 0.009463873348332139, 0.013105869748811616, -0.016198465012629257, 0.001958348251574562, -0.004508188762042858, 0.014160365283578797, -0.012086819418625116, 0.010606981428889755, -0.012379242589246374, -0.003735039946088448, -0.0008263166810330916, 0.004758520810432216, 0.014550262533966294, 0.02171019572190905, -0.02167475144865091, -0.026973810930446616, -0.020983568562343808, 0.0166326680702507, 0.011962761668254, -0.011758951043423177, 0.0013015041586696596, -0.011360192259059872, -0.021373466744053847, 0.007567553971943135, -0.05072210917537375, 0.006065563078590801, 0.002219756751772406, -0.01206023574802024, -0.01611871400081463, -0.005445271532767603, -0.003371726366305827, -0.010669011235397853, -0.008046064233782336, -0.005392103725896581, -0.018218842605050647, 0.006132022720764262, 0.003898974133689418, -0.005671235061215401, -0.011182966236156465, -0.014594568341200242, -0.014594568341200242, -0.008604326904419977, 0.0029596757466400863, 0.0031878544257512, 0.03255643204888777, 0.03473631539022858, -0.029898042636659194, 0.024226806178459983, -0.004656615498016803, -0.006353555482224157, -0.009862632132695443, 0.020646839584488604, -0.022915334540297306, 0.01613643520612116, -0.011918454929697514, 0.007226393761438758, -0.017917557900453584, 0.0076118607104996225, -0.0018719506236168093, -0.0004098353525871069, 0.008692939450210409, -0.0029596757466400863, -0.0018320747684635425, -0.007811240102681274, 0.015073079068700713, -0.013407153522086138, 0.03330078227640462, -0.004049616253157314, 0.020700006925698358, -0.005804154345562322, 0.028710626886222553, -0.005755417305679202, -0.009570208962074184, -0.01590604277333054, 0.01229063004345594, -0.011342470122430802, -0.007106766312394275, 0.021532970630328184, -0.015613618671386742, -0.017979587706961682, -0.003447047542455093, -0.0017611843593021794, 0.0049844838732187435, -0.019069527514987004, -0.019884768151665223, -0.008497991290677931, -0.01398313832935285, -0.01017277743994577, -0.0012583052282754657, 0.019317644878374316, -0.029844873432804363, 0.007700473721951326, 0.017589689525251643, -0.0021621584107441158, 0.027452322589269627, 0.002315015913705964, -0.030217048546562788, 0.008746107722742702, -0.004842703054896017, -0.01710231726377537, -0.010695594906002729, 0.008697370682859583, -0.011493112474729334, 0.01671242094471041, -0.013991999863328655, 0.013336263112924776, 0.0067700368688778, 0.003378372516787681, -0.006482044465244445, -0.011244996042664563, 0.02732826297625343, -0.022933055745603836, -0.095702093346032, 0.017217514411493218, 0.028196670954141398, -0.029171415477093954, 0.00020782527981787647, -0.013123591885440686, 0.0043885613129983755, -0.012095680952600921, 0.0028289716129564816, -0.006163037624018311, 0.008192276284754236, 0.014895853045797306, -0.004762951111758848, 0.004851564123210552, 0.007062460039499058, 0.006118730885461824, 0.02013288365240745, 0.018626461992067214, -0.0042600723299780876, -0.0031834236587632973, 0.009747434984977592, 0.00795302138666527, 0.0018531203295794928, -0.003704025275665034, -0.02238365554026454, 0.016349106433605248, 0.013566657408360477, 0.001561804967120528, 0.014435065386248445, 0.005830738481828468, -0.00028771548069661133, 0.005786431743271981, -0.03457680964130916, -0.002245233079876259, -0.007598568875197185, -0.01654405645578281, 0.006743452732611654, 0.050190428312695906, -0.01040317173538147, 0.005742125470376764, 0.011244996042664563, -0.0067611753349019945, -0.012042512680068628, -0.014576846204571172, 0.011785535645350592, 0.04079744816749275, 0.00020159468836411455, 0.03289316475203314, -0.00831633403512535, -0.0007343806525481195, 0.003976510693332635, -0.007735918926532008, 0.02075317612955319, -0.0216570283806993, 0.00816569261414936, 0.024014134950975895, -0.0004995560557857865, 0.00444615942119603, -0.03434641534587346, -0.01467432028433741, -0.02256088249449049, 0.02844478831752871, 0.010412033269357277, -0.005152848827807616, -0.0035467372385459186, -0.014913576113748916, 0.008422669648867395, -0.0077713641311126895, 0.011431082668221236, -0.015773122557661078, -0.011883009725116831, -0.0015429746730832115, 0.004093922991713801, 0.01680989502447665, -0.02840934218162549, -0.023145726973087925, 0.01263622055528695, -0.006309248743667669, -0.00040125094527485104, 0.016606084399645827, 0.010784207451793162, -0.008653063944303097, -0.001120401156444178, -0.006065563078590801, 0.0028267562294625303, -0.007381466880725192, -0.0038635289291087363, -0.01759855199054999, 0.010669011235397853, 0.008422669648867395, 0.01002213601896978, -0.019477146902003573, 0.007403620250003436, -0.01615415827407277, 0.0031147486330958857, 0.03275138393371041, 0.011803257781979663, -0.010624704496841365, -0.033105834116872145, 0.03813905502997401, -0.0022673864491545024, 0.002831186763619798, -0.005525023475904773, -0.020363277947843154, 0.01229063004345594, -0.01186528665716522, 0.001042864829631596, 0.003227730164489149, 0.0035090766504712856, -0.002532117908177956, -0.021532970630328184, 0.005436410464453068, 0.018342900355421764, 0.005449702299755507, 0.0609303318199002, -0.005711111032783986, 0.0015540513577223332, -0.013566657408360477, -0.00380371497175586, 0.01748335484283214, -0.011324747054479192, 0.005179432498412492, -0.0011314778410832999, -0.015099662739305589, -0.005839600015804274, 0.005436410464453068, -0.00023330153516215604, -0.022419101676167764, -0.0045680024865651, -0.01042089387201054, 0.002964106513627989, -0.027806774635076442, -0.021462080221166822, -0.0250420468151382, -0.013690716090054131, 0.045937003763014114, -0.017793500150082468, -0.002538763825829175, 0.015569311932830255, 0.016889646036291277, -0.005582622049763697, 0.026548468475478437, -0.0033562191475094375, 0.0035400913208946993, 0.008440392716819007, -0.00626051170378455, -0.025556002747219354, -0.011998206872834682, 0.014337591306482206, 0.03048288897790171, 0.012831168714819428, 0.005042082447077668, -0.0010389879667248398, 0.031510798979418934, 0.004109429977679556, -0.014709765488918091, 0.007762503062798154, -0.017439048104275653, 0.011501973077382599, -0.009605654166654865, 0.004368623094383448, -0.010979156542648182, 0.009782880189558273, 0.013690716090054131, 0.0021433280002914816, 0.016490887251927974, 0.0273105417709469, 0.02887013077249689, 0.011298163383874316, 0.007372605346749387, 0.02596362213423592, 0.00035223686603967, 0.0054629941350579445, -0.020629116516536993, 0.009339814666638484, -0.010341142860195914, -0.02071772999364997, -0.0019627790185624645, -0.00700043069865223, -0.00344261677546719, 0.011218411440737147, -0.0024014135416637163, -0.018201119537099037, -0.02280899799523272, 0.013832496908376858, -0.00984490906474383, 0.0009254525312504295, -0.0010345573161522545, -0.008289750364520475, -0.01263622055528695, 0.029454977113739407, 0.004656615498016803, 0.008373932608984275, -0.0055693302144612595, -0.016880785433638013, -0.0020026547573004138, -0.004007525130925414, -0.01105890848578535, 0.02885240770454528, 0.00040346627056114364, -0.00864420241032729, -0.009295508859404536, -0.0014144858064782408, 0.015126247341233006, -0.014585707738546977, -0.03902518607581359, 0.03624273705056882, 0.016260494819137356, 0.014630014477103463, 0.005068666117682545, -0.007545401068326162, 0.00581744664652603, -0.02750548993047938, -0.008994224154807473, -0.0024523659650407875, -0.0036552882357819144, -0.0005122941616300542, 0.023854632927346638, 0.013416015056061945, 0.00344261677546719, 0.005338935919025559, -0.003486923281193042, -0.010491784281171904, 0.019335366083680847, -0.02754093420373752, 0.001042864829631596, -0.003225515013825833, -0.006973846562386084, 0.021054460834150253, 0.00207465297462407, 0.011528557679310016, -0.03245009736646827, -0.004904731930081575, -0.00041481981992935057, -0.010979156542648182, -0.003125825317735007, 0.017456769309582183, -0.0028932158716359906, -0.027434599521318017, -0.010624704496841365, 0.002015946825433487, 0.007523247699047919, 0.01084623725830126, 0.01876824281038994, -0.011324747054479192, -0.02371285210902391, 0.012715972498424119, -0.015480699387039821, 0.013203343828577855, 0.015640202341991618, -0.04232159103313952, -0.011315886451825926, -0.0006972739103469743, 0.0015252520707928706, -0.02633579724799435, 0.0057820014419453485, -0.017385878900420818, -0.0012859968234579526, 0.007532108767362454, -0.03156396632062869, -0.02959675793206213, -0.021763364925763886, -0.007616291477487525, -0.00693397105647877, 0.0018365054190361276, -0.017970725241663335, 0.02933091936336829, -0.008121385875592873, -0.00851128312598037, -0.03145763163820919, -3.4008718232771232e-06, 0.013132453419416492, 0.0035976896619229893, 0.004931316066347721, 0.002390336857024595, -0.017953004036356805, 0.0255028354060096, -0.00336064991449734, -0.011377915327011484, 0.020877233879924306, -0.013912247920191486, -0.0017257390383061803, 0.02073545306160158, 0.006703576761043069, 0.006167467925344943, -0.025786397042655052, -0.01719093074088834, 0.006216205430889333, 0.0020192697842590962, 0.009782880189558273, -0.012122264623205797, -0.014204671090812745, 0.025183827633460925, 0.0013856865195487783, -0.021586137971537935, 0.0027602963544584346, -2.985498104723594e-05, -0.003010628170017157, 0.027186482157930705, 0.0011010170747410325, 0.008489129756702126, 0.001371286876084047, 0.002100129302727923, -0.01602123805840331, -0.01607440726225814, -0.01832517728747015, 0.005086388719972885, -0.012618497487335338, 0.027753605431221608, 0.009330954063985219, 0.015152831011837882, -0.01478951743205526, -0.018874579355454526, 0.022401378608216154, 0.010447478473937958, -0.021036737766198643, 0.007443495755910749, 0.01603896112635492, 0.030589223660321217, 0.0038258683410341034, 0.01811250605998606, 0.023039392290668422, 0.0005743233278539058, 0.00478067371404919, -0.005228169538295612, -0.012033652077415363, 0.005529454242892675, 0.0046964914695853885, 0.0025431943599864425, -0.0004120506778733995, -0.0017800145369241782, -0.003954357324054391, 0.0236951290410723, -0.0030948106473115936, -0.0030770880450212527, 0.004957899736952597, -0.0029995517182086705, 0.010332281326220109, -0.005742125470376764, -0.005879475521711588, -0.030323385091627374, -0.01557817346680606, 0.006149745323054602, 0.013114731282787421, 0.015321195500765484, 0.013593241078965352, -0.0223482112670064, 0.013726161294634814, 0.00150752946850253, -0.008551158631887684, 0.008910041910343672, -0.01419581048815948, -0.017306127888606193, 0.038883405257490865, 0.006659270488147853, -0.0003525137889764139, 0.0002673621832667768, 0.022029203494457725, 0.005795293277247787, -0.025999068270139144, 0.011182966236156465, 0.015330057034741289, -0.003466985528239385, 0.013912247920191486, 0.007842254540274053, 0.00563578985663472, -0.0003851898515011444, -0.007828962704971614, 0.014780655898079455, 0.012494439736964224, 0.03282227248022669, 0.018307456082163623, -0.029773983023642998, -0.023801463723491803, -0.0001507806391439274, -0.01186528665716522, 0.004953469435625964, 0.02162158410744116, 0.009862632132695443, -0.016357968898903594, -0.011422222065567972, -0.009641099371235548, 0.028356174840415735, 0.00017888758962770075, -0.0011696922168237618, 0.005972519300151195, -0.005897198124001929, 0.030695560205385802, 0.015737676421757857, 0.031050012251192617, 0.01603896112635492, -0.0023083697632241096, 0.007221963460112125, -0.003085949346166423, -0.019920212424923363, 0.013531212203779794, 0.02490026599681547, 0.011120937360970909, 0.00956134742809838, 0.005719972101098521, 0.005556037913497551, 0.0015706662682656984, -0.011590586554495574, -0.005542746078195113, 0.005050943515392203, -0.009189173245662492, 0.0030438582239345224, -0.019955658560826584, -0.005906059657977734, -0.01498446652291028, 0.011227272974712953, -0.008573312001165926, 0.01084623725830126, -0.016419996842766613, 0.002505534004742445, -0.0038856820655563448, 0.007151073050950762, -0.01519713775039437, -0.006371278084514497, -0.003340711928713048, -0.009277785791452926, 0.027842218908334583, 0.006911817687200526, 0.003303051340638415, 0.0021012368780595815, 0.018431513832534736, -0.0066548397211599495, -0.0010539413978554002, 0.02910052506793259, -0.029384086704578042, -0.004350900492093108, 0.023766019450233662, -0.008821428433230699, 0.010350003462849179, -0.011679200031608548, -0.013664131488126716, -0.008914472211670304, 0.023074836563926563, -0.022489992085329126, -0.009818325394138955, 0.015223721420999245, -0.023748296382282052, -0.017571966457300033, 0.014071751806465823, 0.0005341705206600657, -0.003296405422987196, 0.004282225699256331, 0.004966761270928402, -0.0026960518629482906, 0.0029862596500755973, -0.0007875484594191417, 0.019973381628778195, -0.002081298892275289, -0.0003176224011653908, -0.009224618450243175, -0.0005660158143745644, 0.0012029221543258097, -0.017164347070283466, 0.0006269372888514403, -0.003225515013825833, -0.0018653045895502727, 0.005901628890989831, 0.013664131488126716, -0.0014499310110589224, -0.005073096884670447, -0.01567564847789484, 0.025449666202154768, 0.002538763825829175, -0.0058883370556873935, 0.0033495732298582186, -0.017687163605017882, -0.01467432028433741, -0.0019306567728073925, -0.026087681747252116, -0.007673890051346451, 0.008737246188766897, 0.013823635374401053, -0.008980932319505036, -0.012529884941544906, -0.01648202664927471, 0.020416445289052905, -0.009782880189558273, -0.009995551417042363, 0.010119610098736019, 0.018910023628712667, -0.009215756916267368, -0.015569311932830255, -0.012131126157181602, 0.004257856713653501, 0.011493112474729334, 0.005387673424569949, -0.024474923541847295, 0.007828962704971614, -0.005024359844787328, -0.008015050261850828, 0.019742987333342496, 0.0020192697842590962, -0.01642885930806496, -0.012955227396513084, 0.019423979560793822, 0.006871941715631942, 0.00969426764376784, -0.022915334540297306, 0.01361982568089277, 0.005445271532767603, -0.023925523336508, 0.0029707524312792078, -0.006548504107417905, -0.007022584067930474, -0.004138229264609018, -0.013380569851481262, -0.015684509080548106, 0.0006767821368968531, 0.012919782191932402, -0.01680989502447665, 0.009277785791452926, -0.007576415505918941, -0.023003946154765198, -0.00489587086176704, 0.0015662356176931133, 0.002735927834516875, 0.013832496908376858, -0.008865735171787185, -0.02013288365240745, 0.006760740298672648, -0.02521927376936415, -0.009960106212461682, 0.014355313443111277, -0.03587056193681039, 0.0021433280002914816, -0.02013288365240745, 0.0045259113643331995, 0.013238789033158536, 0.014922436716402181, 0.006446599260663763, -0.014204671090812745, 0.004120506662318678, -0.02390780026855639, -0.01115638256555159, -0.009756296518953397, 0.01659722379699256, -0.013132453419416492, 0.002494457320103323, -0.019937935492874974, -0.014957881920982862, 0.009951245609808416, -0.021462080221166822, -0.012361520452617304, -0.03278682634432347, -0.0011851995520354688, 0.0068985258518980885, -0.003932203954776148, -0.024031858018927505, -0.008497991290677931, 0.001568450884771747, -0.016225048683234135, -0.010376588064776595, 0.0007194271632099003, 0.022489992085329126, -0.0021400050414658723, 0.007802378568705468, -0.006973846562386084, -0.0005361089521134437, -0.007190948556858076, 0.016730144012662023, 0.001999331798474804, 0.016030100523701657, -2.943960755605608e-05, -0.02052278183411749, -0.0029796137324243784, -0.005954796697860854, -0.013034979339650253, 0.013495766999199113, 0.007709335255927131, -0.00979174172353408, -0.012122264623205797, -0.004647754429702268, -0.00021737887375299197, 0.01198048380488307, 0.008112524341617067, 0.002430212828593179, -0.0023393844336475236, 0.013291957305690829, 0.00579972404423569, 0.010624704496841365, 0.0009165912301052591, -0.00816569261414936, 0.020505058766165877, 0.014116058545022311, 0.008870166404436358, -0.00934867620061429, 0.009720851314372716, 0.012476716669012613, -0.01765171933175974, 0.009144866507106005, -0.006942832124793305, -0.0063358328799338155, -0.006087716447869045, 0.004523696213669883, -0.006699146459716437, 0.0008667463820598463, -0.003856882778626882, 0.013646409351497645, -0.0019173648210896368, 0.01609212846756467, 0.0050376516800897655, 0.004282225699256331, 0.003522368718604359, 0.009729712848348521, 0.0022950779279216718, -0.004829411219593579, 0.01657950072904095, 0.0006845358045027066, -0.009109421302525324, -0.010863959394930331, -0.02233048819905479, 0.0024124902263028382, 0.0016216188080580868, -0.014036306601885143, 0.004273364165280525, -5.20255506886551e-05, 0.0024922419366093717, 0.006069993845578704, 0.01263622055528695, 0.007691612653636791, -0.020363277947843154, 0.007053598505523253, -0.003653072852287963, 0.026229462565574842, -0.0027204206157204858, -0.0006717976404507801, 0.014266700897320843, 0.006251650635470014, -0.0030128435535111083, -0.016402275637460082, 0.030092990796191675, -0.00797074352329434, 0.015108524273281394, -0.0024279974450992278, -0.0035157225681225045, 0.012919782191932402, 0.013531212203779794, -0.01696939891075099, 0.0003253760396674149, -0.003912266201822491, 0.009224618450243175, 0.0263889645892041, 0.012742556169028994, 0.011519696145334211, -0.008245443625963987, 0.015374363773297777, -0.013203343828577855, -0.013291957305690829, -0.010606981428889755, 0.019565760379116545, 0.0026185155361357084, 0.0013347339797563898, -0.009525902223517697, -0.013123591885440686, 0.016242771751185745, 0.0234647347456366, -0.02723965136178554, -0.004922454532371915, 0.003289759272505342, 0.007089043710103934, -0.04058477507736358, -0.0006047840359885143, 0.01342487659003775, 0.004714214071875729, -0.0049623305039405, -0.008028342097153265, 0.000808040232869263, 0.006428876658373423, 0.0037771310683203485, 0.019742987333342496, 0.009455011814356333, -0.00019951780999427922, -0.0008124708834418482, -0.01298181106711796, -0.0045259113643331995, 0.017226376876791565, 0.0006839819586292187, 0.0003378372516787681, -0.021515247562376574, -0.007350452443132413, -0.01832517728747015, -0.020150606720359062, 0.009995551417042363, 0.007319437539878364, -0.016978259513404252, 0.019477146902003573, 0.010217084178502258, -0.006579518545010684, -0.02055822610737563, 0.005892767822675296, 0.011360192259059872, 0.003340711928713048, -0.009295508859404536, 0.017899834832501973, -2.076868308550562e-07, 0.01217543289573809, 0.03927330157655582, 0.012875475453375916, 0.008653063944303097, 0.004705353003561194, -0.014993327125563545, 0.014851546307240818, -0.0011774458262219566, -0.009933522541856804, -0.004785104481037092, -0.0056003446520540385, -0.01140449899761636, 0.015436392648483333, 0.031085456524450758, 0.01361982568089277, -0.008382794142960082, 0.015702232148499716, 0.020363277947843154, 0.029384086704578042, -0.00824987485861316, -0.007412481318317971, 0.007040306670220814, 0.00885687363781138, -0.017687163605017882, 0.004025247733215755, -0.02121396285777951, -0.0029685370477852565, -0.0028112490106661408, -0.0010506184390297907, 0.0014543616616315075, 0.0223482112670064, -0.004277794932268429, -0.0026517453572224388, -0.0085777432338151, 0.00016808787158106698, 0.005179432498412492, 0.03312355532217868, -0.015188276216418562, -0.00048072579085229944, 0.00011166628568691968, 0.011998206872834682, 0.01217543289573809, 0.019796154674552247, -0.012990672601093765, 0.02373057331433044, 0.009729712848348521, 0.005316783015408586, -0.019884768151665223, 0.004714214071875729, 0.011466527872801918, -0.009233479052896438, -0.0014565769287101414, -0.01705801238786396, 0.013903387317538221, 0.013088146680860004, 0.0030770880450212527, 0.002906507939769064, -0.007040306670220814, -0.018413790764583125, 0.009649960905211353, -0.03353117657184033, -0.010837375724325455, 0.003774915684826397, -0.0004386345813089107, 0.014257839363345038, -0.01138677686098729, 0.011626031759076255, 0.010110748564760214, 0.01361982568089277, 0.014311006704554789, 0.034435028822986435, 0.02591045479302617, -0.012113403089229991, 0.022277320857845038, -0.0022419101210506493, -0.020575949175327242, -0.0003574982563186576, 0.0091360049731302, -0.011298163383874316, 0.01519713775039437, 0.04497998417065825, -0.0036087663465621113, -0.009720851314372716, -0.020646839584488604, 0.008245443625963987, -0.003907835434834589, -0.0072086711591484165, -0.014054029669836753, -0.016765588285920164, -0.00301505893700506, 0.006623825283567171, 0.012450132998407736, -0.012024790543439558, 0.027381432180108262, 0.016721281547363676, 0.008382794142960082, 0.003001766868871987, 0.006357986249212059, -0.016535193990484462, 0.012972950464464695, -0.0005820769372521004, 0.005618067254344379, 0.011182966236156465, -0.0020613609064909968, 0.014488232727458196, -0.001539651714257602, 0.013247650567134343, 0.0021809885883661145, 0.0020580379476653875, 0.008746107722742702, 0.008799275995274995, 0.011820980849931275, -0.004891440094779136, -0.019991102834084725, 0.013451460260642626, -0.011838702986560345, 0.009649960905211353, 0.01844923690048635, 0.0005280783906746757, -0.002164373561407432, 0.018980914037874032, -0.00969426764376784, -0.001978286237358854, 0.009800603257509885, 0.0019184725128366126, -0.011555141349914892, -0.005901628890989831, -0.015666786012596495, -0.003867959463266004, -0.008094802204987997, 0.009508180086888626, -0.004029678500203657, 0.003998663596949608, -0.008400517210911692, 0.013974277726699584, -0.033513455366533794, 0.008258736392588966, 0.01165261542968113, -0.003861313545614785, -0.016960536445452642, -0.023163450041039535, 0.010863959394930331, 0.02729281870299529, 0.02732826297625343, 4.479112450313249e-05, -0.017642856866461398, 0.02006199324324609, -0.004235703810036528, 0.018910023628712667, 0.01002213601896978, 0.004233488193711941, 0.0007493340836786798, 0.013469183328594238, -0.0011741228673963471, -0.011041185417833738, 0.0182720099462604, -0.0005554930338165893, -0.012086819418625116, 0.0015474053236557968, -0.009738573451001787, -0.015073079068700713, -0.014869269375192428, -0.010775346849139897, 0.004687630401270853, 0.021089905107408394, 0.004727505907178167, 0.02052278183411749, -0.00885687363781138, 0.013185621691948785, 0.007682751119660986, -0.014505955795409808, -0.009419566609775653, -0.021019014698247032, -0.006185190527635284, 0.004709783304887826, 0.0019561331009112456, -0.01828973301421201, -0.00410721482701624, 0.015773122557661078, -0.0020668992488105577, -0.006845358045027066, -0.01398313832935285, 0.001905180561118857, 0.004940177134662257, 0.0021145287133620193, -0.005316783015408586, 0.003506861266977334, -0.007594138108209281, 0.00956134742809838, 0.012645082089262755, 0.018856856287502916, -0.016278216024443886, 0.024191361905201842, -0.00609657751618358, 0.01853784851495424, -0.010093026428131143, -0.0033429270793763643, -0.029827152227497832, 0.0036730108380722552, -0.010429755405986348, 0.0117323673728183, -0.015276889693531538, 0.02144435715321521, -0.008130247409568678, 0.004492681310415834, -0.020434168357004515, 0.002702698013430145, 0.00045026505361386155, 0.006645978652845414, -0.02254315942653888, -0.014993327125563545, -0.009392982939170777, -0.01317676015797298, 0.016783311353871774, 0.006712438295018875, -0.007820101170995809, -0.0005876152795716614, 0.002622946070292976, -0.010766485315164092, 0.014204671090812745, 0.022667217176909994, -0.0005610313761361501, 0.015117385807257199, -0.004364192327395546, -0.013362847714852192, 0.011501973077382599, 0.024510367815105436, 0.012370381055270567, 0.0058351692488163714, 0.006752314266587459, -0.019955658560826584, 0.012831168714819428, -0.0019406257656995386, 0.006579518545010684, 0.004317670903837013, 0.0063358328799338155, -0.0015374363307636505, 0.0091360049731302, -0.003116964016589837, 0.007731488159544105, 0.018608738924115603, 0.001726846730053156, 0.012902060055303331, -0.021373466744053847, -0.00874167742141607, -0.010651288167446243, 0.00400974028158873, 0.00244793543088352, 0.014621152943127658, -0.01027025245103455, 0.007598568875197185, 0.006069993845578704, 1.8674506695504647e-05, -0.009100559768549519, -1.2192947463855877e-05, 0.0019805016208528053, 0.005578191282775795, 0.010908266133486819, -0.009862632132695443, -0.0035245838692676747, -0.023659682905169076, -0.009242340586872245, 0.0182720099462604, 0.007868838210878928, -0.0022120032587895286, 0.022029203494457725, 0.01454140099999049, -0.0005485701641247969, -0.010917127667462624, -0.0004563571835992515, -0.019299921810422706, -0.015728815819104593, 0.016588363194339296, 0.0035423064715580156, -0.004333177889802767, 0.0013557796572876576, 0.008475837921399687, -0.006849788346353698, 0.0034116023378744113, -0.008591035069117538, 0.005480716737348285, 0.001839828377861737, 0.011032323883857933, -0.005502870106626529, 0.006433307425361325, 0.009025239058061523, -0.005201585867690735, -0.007483371727479334, 0.011820980849931275, 0.0007144426667638272, 0.01242354932780286, 0.020629116516536993, -0.004486035392764615, 0.011679200031608548, 0.012086819418625116, -0.026318074180042735, 0.02596362213423592, -0.0025653477292646863, 0.006734591664297119, -0.017439048104275653, -0.0020081930996199747, -0.0288878538404485, -3.7037483272624395e-06, 0.00711562784637008, 0.018342900355421764, -0.006765606101889897, 0.015321195500765484, -0.008794844762625821, 0.021568414903586325, 0.006552934874405808, -0.003506861266977334, -0.004545849117286857, -0.01061584296286556, 0.014293284567925718, -0.007682751119660986, -0.006858649880329504, 0.004512619529030762, -0.01184756452053615, -0.010651288167446243, 0.0023571070359378644, -0.024492646609798906, 0.002786880257893946, -0.0025808549480610754, -0.006929540289490867, 0.008927764046972743, 0.017988448309614945, 0.004441729119869398, 0.0045901558558433435, 0.012237461770923648, 0.006783328704180238, -0.011767812577398982, -0.007443495755910749, -0.011014601747228863, -0.00010502030982804188, -0.005985811135453632, 0.01661494686494417, 0.008373932608984275, -0.012733694635053189, 0.009144866507106005, -0.00882585966587987, -0.0270801474755112, 0.01609212846756467, -0.017439048104275653, 0.0040163861992399495, 0.00019024113899590765, 0.005719972101098521, 0.005808585112550225, -0.01719093074088834, 0.015152831011837882, -0.0055870528167516, -0.012104542486576726, 0.011129798894946714, 0.01163489329305206, -0.008515713427307002, -0.027399153385414793, -0.011962761668254, -0.005263615208537563, -0.02688519931597872, -0.015914903375983804, -0.029915763841965724, 0.01894546976461589, -0.002569778263421954, 0.023553348222749574, 0.01832517728747015, 0.008746107722742702, 0.009960106212461682, -0.00451926544668198, 0.006778897937192335, -0.00028176179762767864, -0.01296408893048889, 0.0016149727739915502, -0.0011287086699235194, -0.003198930877559687, 0.03005754466028845, 0.0011386776628156655, 0.0042401341113631605, 0.010766485315164092, 0.005582622049763697, 0.015374363773297777, 0.009065114563968836, 0.03984042484984673, -0.006256080936796647, -0.005068666117682545, 0.013805912306449442, 0.017403601968372432, -0.0012350442836655642, -0.005090819486960788, -0.0013469183561424871, 0.007585276574233476, 0.014311006704554789, 0.02282672106318433, 0.007598568875197185, 0.008072648835709753, 0.014027445067909336, -0.015330057034741289, -0.025786397042655052, 0.012840030248795233, 0.026530745407526826, 0.007319437539878364, -0.012529884941544906, 0.009951245609808416, -0.011510834611358404, 2.672237217984048e-05, -0.013416015056061945, 0.008272028227891404, 0.013779328635844565, 0.010872820928906136, 0.021036737766198643, -0.0025409792093231266, 0.006362416550538692, 0.015799706228265955, 0.0042622874806414034, 0.010287974587663621, -0.00502879061177523, -0.012219738702972036, 0.006858649880329504, -0.005201585867690735, 0.013034979339650253, -0.006340263646921719, 0.005325644083723121, 0.01544525418245914, 0.006344693948248351, 0.004459451722159739, 0.015161692545813687, 0.013478044862570043, -0.006065563078590801, 0.0013181190692130246, -0.025662337429638856, 0.01038544866742986, 0.00043337319102992313, -0.008688509148883777, 0.007935298318713658, 0.005644650924949255, 0.0021100981792047517, -0.004980053106230841, -0.00795302138666527, -0.009100559768549519, -0.015533866728249574, -0.0221178169715707, 0.012671665759867631, 0.021816532266973637, -0.007687181886648888, -0.03112090266035398, 0.008701800984186216, -0.005108542089251129, 0.0014299930252746303, -0.02238365554026454, 0.016588363194339296, -0.03241465123056504, -0.008174553216802624, -0.013274234237739218, -0.002202034149482065, -0.008223291188008285, -0.016703558479412066, 0.007585276574233476, 0.025786397042655052, 0.005374381123606241, 0.00895434864890016, 0.009144866507106005, 0.007589707341221379, -0.016145297671419506, 0.0051750017314245896, -0.00711562784637008, 0.001549620707149748, -0.005378811890594143, 0.004257856713653501, 0.02123168592573112, 0.025361054587686876, -0.025006602541880057, 0.007518816932060015, 0.02117851858452137, -0.0027004826299361936, 0.009729712848348521, 0.0021887423141796272, -0.00700043069865223, -0.0029042925562751125, -0.005578191282775795, 0.011005740213253058, 0.02186969960818339, 0.00688523355093438, -0.0007803486376867761, 0.025857287451816417, -0.00683649651105126, -0.009738573451001787, 0.00969426764376784, -0.018342900355421764, -0.0029175846244081857, 0.017554245251993502, -0.023482457813588212, 0.00324545276677949, -0.04019487875829862, -0.0032432376161161736, 0.015055356932071643, 0.007802378568705468, 0.011448805736172848, 0.014621152943127658, 0.009446151211703068, 0.003710671426146888, -0.029508144454949158, -0.0029862596500755973, -0.00851128312598037, 0.02302166922271681, 0.010766485315164092, 0.0012959658163500987, -0.009889215803300318, -0.013779328635844565, 0.005821876947852663, 0.02553827967926774, 0.016889646036291277, 0.005068666117682545, 0.003987586912310486, -0.004616739526448219, -0.010766485315164092, -0.005228169538295612, 0.0044217909012544705, -0.001933979731633002, -0.004572433253553002, 0.019051804447035393, 0.010376588064776595, -0.015764260092362734, -0.004317670903837013, 0.005999103436417341, -0.0036287043323464034, 0.00047186446060329965, -0.05944163136486649, 0.006561795942720343, 0.010899404599511013, 0.013291957305690829, -0.015392085909926847, -0.001189630202608054, 0.008639772109000658, 0.0082631666939156, 0.007890991580157172, 0.00481611891862987, -0.005666804294227498, -0.026495301134268685, -0.005560468680485454, -0.011812119315955468, -0.0017988448309614947, -0.011812119315955468, -0.029490421386997548, -0.0022751399421373796, 0.005090819486960788, 0.010908266133486819, -0.0023371690501535723, -0.01684533929773479, -0.0010135117550363042, -0.018103645457332798, 0.0006402292405691958, 0.009295508859404536, -0.007868838210878928, -0.01605668419430653, -0.010917127667462624, -0.018360623423373374, 0.009153728041081811, -0.0007554262136640697, -0.005808585112550225, 0.025166104565509315, -0.0020668992488105577, -0.006291526141377329, -0.006353555482224157, -0.004457236105835152, -0.004731936674166069, -0.023039392290668422, -0.01115638256555159, 0.013141314953392297, -0.01296408893048889, -0.023766019450233662, 0.01610099093286302, 0.01309700821483581, -0.003504646116314018, 0.017182070138235077, 0.014647736613732533, -0.007527678466035821, -0.002862201434043212, 0.010456339076591223, 0.0032299455479831004, 0.02123168592573112, 0.014505955795409808, -0.0038369450256732253, 0.013610964146916963, 0.018679629333276968, -0.007425773153620409, 0.01322992749918273, 0.004182536003165506, -0.014957881920982862, -0.010890543065535207, 0.01375274496523969, 0.002877708652839601, 0.010075303360179531, -0.009942384075832611, 0.006911817687200526, 0.010686733372026923, 0.009880354269324513, -0.007439065454584118, -0.01263622055528695, 0.008365072006331012, -0.03693391621158575, 0.009330954063985219, 0.02117851858452137, -0.009029669359388155, -0.00984490906474383, 0.004652184731028901, 9.20744972293015e-05, 0.012352658918641497, 0.0010423109255504493, 0.023145726973087925, -0.02486482172355733, -0.009127143439154394, 0.02080634347076294, -0.009025239058061523, 0.014727488556869704, 0.01127157971326944, 0.005821876947852663, 0.02034555487989154, 0.0111741056335032, 0.0004624493426884708, -0.002221972135266357, -0.015055356932071643, -0.004709783304887826, 0.0030881647296603742, -0.0043575464097443265, 0.02277355372197458, -0.004727505907178167, -0.01038544866742986, -0.03152852018472547, 0.007004861465640133, 0.007567553971943135, -0.0028843545704908204, -0.02229504206315157, -0.024598981292218408, 0.00303942745694662, -0.012583052282754657, 0.0011840918602884932, -0.0009847124681068417, -0.008187845983427602, 6.126761359589104e-05, 0.010323419792244302, -0.01513510794388627, -0.003041642840440571, 0.001549620707149748, -0.011094353690366031, 0.011891871259092638, -0.0054629941350579445, 0.001990470497329634, -0.017509938513437018, 0.006007964504731876, 0.007700473721951326, 0.0011325855328302755, -0.005436410464453068, 0.0013779329101505835, -0.017182070138235077, -0.009242340586872245, -0.019406256492842208, -0.025396498860945017, 0.012645082089262755, -0.01038544866742986, -0.0061541760900425055, -0.00982718692811476, -0.019955658560826584, 0.02729281870299529, 0.0023947676240124974, 0.019813877742503858, -0.013469183328594238, -0.0057598480726671055, -0.01642885930806496, 0.02435086579147618, -0.006132022720764262, -0.03003982345498192, 0.024209084973153452, 0.01107663062241442, -0.005121833924553567, -0.017846667491292222, -0.030252494682466012, -0.010243667849107133, 0.0021211748638438736, 0.033212168799291654, -0.013017256271698642, 0.02318117310899115, -0.014107197011046506, -0.010739901644559216, -0.01710231726377537, 0.022259597789893427, 0.0057598480726671055, -0.0008534544303420906, 0.025396498860945017, -0.03090823143286989, -0.0042600723299780876, -0.012113403089229991, -0.01583515236416918, 0.015126247341233006, 0.02330523085936226, 0.0005314013495002853, -0.004326531972151548, -0.00874167742141607, 0.011041185417833738, -0.004585725088855441, -0.024474923541847295, -0.017607412593203257, -0.015507283057644696, 0.0044306524352302765, -9.498210802958187e-05, -0.018874579355454526, 0.014116058545022311, 0.013052701476279323, 0.01048292367851864, -0.016340245830951984, -0.00681877390876092, 0.002043638304200656, -0.005347797453001364, -0.005746556237364667, -0.0010046504538911338, 0.023606515563959325, 0.0023703991040709377, 0.00806821760306058, -0.0037217478779553747, -0.011262718179293633, 0.008848013035158114, -0.009933522541856804, 0.002245233079876259, 0.009517041620864431, 0.022897611472345696, -0.0081169546429437, 0.0020702222076361674, -0.012689387896496701, -0.0024700885673311283, 0.0033517883805215345, 0.029490421386997548, 0.0037904231364534217, -0.018236565673002258, -0.049197962584436816, -0.016730144012662023, -0.009658822439187158, -0.00982718692811476, 0.013416015056061945, -0.007035875903232912, -0.003967649159356829, 0.0187327985371318, 0.0007377036113737289, -0.010908266133486819, -0.017997308912268212, -0.0017944141803889096, 0.014275561499974108, -0.00069838160209395, -0.01240582625985125, 0.009915799473905194, -0.003905620051340637, 0.002123390247337825, 0.02186969960818339, -0.009073976097944643, 0.023340676995265486, -0.022933055745603836, -0.003874605380917223, 0.0004948485113802868, 0.0026185155361357084, 0.007283992335297682, 0.008201137818730041, 0.0007471187583923872, -0.0008362856157175789, -0.016260494819137356, -0.009818325394138955, -0.016703558479412066, -0.004603447691145781, 0.005737694703388862, 0.007173225954567735, 0.013708438226683202, -0.00925120212084805, 0.0008495775674353345, 0.005959227464848757, -0.033513455366533794, 0.003659718769939182, 0.008834721199855677, 0.017377018297767555, 0.02326978658610412, -0.02124940899368273, 0.005657943225912963, -0.012485578202988418, -0.0008722846661717484, 0.0019295490810604166, -0.014931298250377987, 0.0015130678108220908, -0.009224618450243175, 7.725949846328374e-05, -0.024492646609798906, 0.004793966015012897, 0.0006352448023307817, 0.004215765591421601, 0.01728840482065458, -0.013717299760659009, 0.00778908673340303, -0.008329626801750329, -0.012458994532383543, -0.01592376397863707, 0.005560468680485454, 0.00530792148143278, 0.009906938871251929, -0.01603896112635492, -0.02163930531274769, 0.02817894974883487, 0.010704456439978534, -0.0012139987225496139, 0.010491784281171904, -0.007514386165072113, -0.02571550663349369, 0.011918454929697514, -0.013522350669803989, -0.010589259292260685, 0.003520153335110407, -0.0015850659117304297, -0.017013705649307473, 0.020150606720359062, -0.007576415505918941, 0.03496670968566428, 0.020575949175327242, 0.014222394158764355, 0.007416912085305874, -0.007248547130717001, 0.022224151653990203, 0.02569778356554208, 0.000644106103475952, -0.0019295490810604166, 0.026920643589236862, 0.021816532266973637, 0.023429290472378458, -0.013345124646900582, 0.004373053861371351, -0.01853784851495424, 0.009304369462057802, -0.03845806280252268, -0.005427548930477263, 0.006756745033575362, 0.04558254985456222, 0.01186528665716522, -0.0021134211380303613, 0.017846667491292222, -0.026566191543430047, -0.021834255334925248, -0.008422669648867395, -0.009534763757493502, -0.026654805020543022, -0.0006031225565757096, 2.8262715997713136e-05, 0.00586618368640915, 0.017793500150082468, 0.011324747054479192, -0.00507752765165835, -0.001517498461394676, -0.0006878587633283161, 0.015259166625579926, 0.025999068270139144, 0.007762503062798154, 0.012228600236947842, -0.009260063654823856, -0.00489587086176704, 0.00870623221683539, 0.013903387317538221, -0.03110317959240237, 0.006021256340034314, -0.007532108767362454, 0.025680060497590466, 0.0029220151585654533, -0.0013181190692130246, -0.004541418815960224, 0.012299490646109204, -0.006114300118473921, -0.023358400063217096, -0.01375274496523969, -0.0053566585213158996, 0.0016526333620661832, 0.01467432028433741, -0.005436410464453068, -0.022649494108958383, -0.0014853762156396039, 0.00044832662216048343, 0.010030996621623045, -0.023801463723491803, 0.022897611472345696, 0.026211739497623232, 0.01917586406005159, -0.009215756916267368, -0.006185190527635284, 0.003314128025277537, -0.010509507349123516, 0.018307456082163623, 0.03027021588777254, 0.027452322589269627, -0.004042970335506096, -0.002877708652839601, 0.013859080578981733, -0.007696042954963424, -0.0011453235804668844, -0.006052271243288363, -0.015418670511854263, 0.005485147504336188, 0.007022584067930474, -0.011147521031575784, 0.010908266133486819, -0.002207572491801626, -0.007133350448660421, -0.010704456439978534, -0.0007703797030022888, -0.017820083820687345, -0.0018365054190361276, 0.01796186463901007, 0.06302159795883787, -0.014514817329385614, -0.0056003446520540385, 0.02034555487989154, -0.004793966015012897, -8.16209255776403e-05, 0.0025033186212484937, 0.010695594906002729, -0.010234807246453868, 0.00642444589138552, 0.001928441389313441, -0.020505058766165877, 0.0011874148191141026, 0.00948159641628375, -0.002844478831752871, -0.005680096129529936, 0.014098335477070699, -0.00022900934605794282, 0.010589259292260685, -0.0018708429318698334, 0.0248116525197025, 0.0182720099462604, -0.00489587086176704, 0.0005593698385156867, -0.0009631130611174036, 0.008272028227891404, -0.009906938871251929, 0.003163485672979005, -0.01702256625196074, 0.02073545306160158, 0.010722178576607604, 0.00264509943957122, -0.0028267562294625303, 0.0008019480446762143, -0.005476286436021653, -0.0111741056335032, -0.01005758122355046, 0.017793500150082468, 0.012645082089262755, -0.014647736613732533, -0.014824962636635942, -0.00906954579661801, -0.010190500507897382, -0.005985811135453632, -0.001774476311019935, 0.007753641528822348, -0.011874148191141026, 0.003283113354854123, -0.008821428433230699, -0.007257408664692807, -0.006712438295018875, 0.010350003462849179, -0.03296405329854942, 0.012804585044214552, -0.012272906975504328, -0.01163489329305206, 0.003850236860975663, 0.03455908843600263, -0.01082851419034965, 0.013478044862570043, 0.008427100881516568, -0.008914472211670304, -0.015870596637427316, 0.017049149922565614, -0.03089050836491828, 0.0005458009929650166, -0.03314128025277537, -0.01003985815559885, 0.0006125377035943678, -0.006238358334506307, -0.0012472286600516615, -0.009729712848348521, 0.0029707524312792078, 0.002189849889511285, 0.00080305573642319, -0.01388566424958661, 0.018103645457332798, -0.002485596018958153, -0.0038014998210925436, -0.014594568341200242, -2.6410841151960916e-05, -0.0066548397211599495, 0.00874167742141607, 0.005369950822279608, 0.017314988491259457, -0.020381001015794764, 0.011315886451825926, 0.0026871905618031204, -0.018342900355421764, -0.018378346491324984, 0.0340805786398247, 0.013433738124013555, 0.0018176751249988111, 0.02238365554026454, 0.011679200031608548, -0.015152831011837882, 0.004455020955171836, 0.009853770598719636, 0.0030327815392954005, 0.0010572643566810096, 0.008090370972338824, 0.009782880189558273, -0.008227721489334917, 0.007478940960491431, 0.011067769088438616, 0.01465659814770834, 0.01765171933175974, -0.020221497129520427, -0.005808585112550225, -0.0012926428575244892, 0.01696939891075099, 0.007656167449056109, -0.015143969477862076, 0.010917127667462624, 0.005232600305283514, -0.00800175842654839, 0.005436410464453068, 0.01968981812948766, -0.00048654099790094546, 0.006592810845974392, 0.006415584357409714, 0.0060478404763004605, 0.009051822728666399, -0.003112533249601934, -0.0036087663465621113, -0.013265372703763413, -0.0040053095146008275, -0.023003946154765198, 0.010225945712478063, 0.016357968898903594, 0.007966313221967709, 0.014647736613732533, 0.0029951209512207675, -0.020150606720359062, -0.03041199856874035, 0.0010799715136250822, 0.007554262136640697, 0.010872820928906136, 0.016357968898903594, 0.017837806888638955, -0.0020668992488105577, 0.0006092146865610995, 0.006291526141377329, -0.01400086139730446, 0.010279113053687816, -0.019406256492842208, -0.014647736613732533, 0.005826307714840565, -0.016951675842799375, -0.010917127667462624, 0.012361520452617304, 0.005826307714840565, -0.017261821150049705, -0.001418916457050826, -0.009100559768549519, 0.006845358045027066, -0.008121385875592873, 0.010101887030784407, -0.021586137971537935, 0.00042063505608182596, 0.0007050275779528278, -0.0031856390422572486, 0.0063757088515024, 0.018856856287502916, -0.007563123670616503, 0.0182720099462604, 0.008046064233782336, 0.009322092530009414, 0.0011342470122430804, 0.0332653361405014, -0.0012715971799932212, -0.012972950464464695, 0.01344259965798936, -0.005392103725896581, 0.02233048819905479, -0.0008927764396218696, -0.003613196880719379, -0.0030505041415857413, 0.00015174984031870173, -0.003934419105439464, 0.015959210114540292, -0.011714645236189229, 0.0012206447566161503, 0.008134677710895311, -0.0329286071626462, -0.010527230417075126, -0.011218411440737147, -0.0041648134008751645, -0.0006529674046211224, -0.015533866728249574, -0.028746073022125774, -0.003030566155801449, -0.016171881342024384, 0.003887897449050296, 0.002729281916865656, -0.0019949012643175365, -0.011590586554495574, 0.010048719689574655, -0.03624273705056882, 0.04412929553543173, 0.01096143347469657, -0.0010018812827313536, -0.016340245830951984, -0.01805933871877631]
